In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files areavailable in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/quora-insincere-questions-classification/train.csv
/kaggle/input/quora-insincere-questions-classification/sample_submission.csv
/kaggle/input/quora-insincere-questions-classification/test.csv
/kaggle/input/quora-insincere-questions-classification/embeddings/glove.840B.300d/glove.840B.300d.txt
/kaggle/input/quora-insincere-questions-classification/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec
/kaggle/input/quora-insincere-questions-classification/embeddings/paragram_300_sl999/README.txt
/kaggle/input/quora-insincere-questions-classification/embeddings/paragram_300_sl999/paragram_300_sl999.txt
/kaggle/input/quora-insincere-questions-classification/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin


In [2]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from gensim import models

import re
from collections import Counter
import gensim
import heapq
from operator import itemgetter
from multiprocessing import Pool

In [3]:
data = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
display(data.head())
data.target.value_counts

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


<bound method IndexOpsMixin.value_counts of 0          0
1          0
2          0
3          0
4          0
          ..
1306117    0
1306118    0
1306119    0
1306120    0
1306121    0
Name: target, Length: 1306122, dtype: int64>

In [4]:
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer() 

def lower_token(tokens): 
    return [w.lower() for w in tokens]   

def lemmatize_words(tokens):
    return [lemmatizer.lemmatize(word) for word in tokens]

def remove_stop_words(tokens): 
    return [word for word in tokens if word not in stop_words]

#https://towardsdatascience.com/nlp-learning-series-part-1-text-preprocessing-methods-for-deep-learning-20085601684b 

def clean_numbers(sen):
    res = []
    for word in sen:
        if bool(re.search(r'\d', word)):
            word = re.sub('[0-9]{5,}', '#####', word)
            word = re.sub('[0-9]{4}', '####', word)
            word = re.sub('[0-9]{3}', '###', word)
            word = re.sub('[0-9]{2}', '##', word)
            word = re.sub('[0-9]{1}', '#', word)
        res.append(word)
    return res


word2vec_path = '../input/quora-insincere-questions-classification/embeddings/\
GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)
words = word2vec.index2word

In [5]:
import time
from tqdm import tqdm_notebook

def process_basic(data, lemmatize=False):
    tokens = [tokenizer.tokenize(sen) for sen in data['question_text']]
    lower_tokens = [lower_token(token) for token in tokens]
    lemmatized_tokens = lower_tokens
    if lemmatize:
        lemmatized_tokens = [lemmatize_words(token) for token in lower_tokens] 
    filtered_nums = [clean_numbers(sen) for sen in lemmatized_tokens]
    no_stopwords = [remove_stop_words(sen) for sen in filtered_nums]
    data['basic'] = [' '.join(sen) for sen in no_stopwords]
    return data


In [6]:
data = process_basic(data)
display(data.head())

,qid,question_text,target,basic
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0,quebec nationalists see province nation ####s
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0,adopted dog would encourage people adopt shop
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0,velocity affect time velocity affect space geo...
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0,otto von guericke used magdeburg hemispheres
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0,convert montra helicon mountain bike changing ...


In [7]:
lemmatizer.lemmatize('do')

'do'

In [9]:
display(data.head(50))

NameError: name 'train_data' is not defined

In [ ]:
data.to_csv('processed_train_lemmatized.csv')